In [1]:
# Dependencies
using Base
using StaticArrays

using BenchmarkTools

# Forward Mode Automatic Differentiation

**Things to do**

- `Dual` Number Intro
- `Dual` Number Example
- Why we need `MultiDual`
- `MultiDual` Intro
- `MultiDual` Example
- Stencil Computation: Where Parallelism comes in...

## Introduction
The core idea behind Forward Mode Automatic Differentiation comes from **Complex Step Finite Difference** where the value and the chance is stored separately, i.e. *Dual Numbers*. From Taylor Series we know that $f(x+h) \approx f(x)+hf^{'}(x)$. This statement gives us a way to represent a function by it's value and derivative:

$$
f(x)\rightarrow f(x)+\epsilon f^{'}(x) \tag{1}
$$

To enable this idea, we can define a custom `struct` which stores both value and the derivative. This `Dual` number is a multidimensional number where senstivity (or derivative) of the function is propagated alongside the evaluation.

In [2]:
struct Dual{T}
    # Value
    val::T
    # Derivative
    grad::T
end

Using Equation 1, let's define two function $f(x)$ and $g(x)$:

$$
f(x)\rightarrow f(x)+\epsilon f^{'}(x) \tag{2}
$$

$$
g(x)\rightarrow g(x)+\epsilon g^{'}(x) \tag{3}
$$

Now using the two above defined equations, we can perform several arithmatic operations on them. Adding Equation 2 and 3 gives 

$$
(f+g)(x)\rightarrow \big(f(x)+g(x)\big)+\epsilon \big[f^{'}(x)+g^{'}(x)\big]
$$

This can be coded easily by overriding `+` operator such that it collects gradients as well as the value inside the `Dual` type.

In [3]:
function Base.:+(f::Dual{T}, g::Dual{T}) where {T}
    return Dual{T}(f.val+g.val, f.grad+g.grad)
end

function Base.:+(f::Number, g::Dual{T}) where {T}
    return Dual{T}(f+g.val, g.grad)
end

function Base.:+(f::Dual{T}, g::Number) where {T}
    return Dual{N,T}(f.val*g, f.grad)
end

Similar rules can be applied for all the common operators.

- **Subtraction Rule**

    $$(f-g)(x)\rightarrow f(x)-g(x)+\epsilon \big[f^{'}(x)-g^{'}(x)\big]$$

In [4]:
function Base.:-(f::Dual{T}, g::Dual{T}) where {T}
    return Dual{T}(f.val-g.val, f.grad-g.grad)
end

function Base.:-(f::Number, g::Dual{T}) where {T}
    return Dual{T}(f-g.val, g.grad)
end

function Base.:-(f::Dual{T}, g::Number) where {T}
    return Dual{T}(f.val-g, f.grad)
end

- **Multiplication Rule**

    $$(f \cdot g)(x)\rightarrow f(x)g(x)+\epsilon \big[f(x)g^{'}(x)+f^{'}(x)g(x)\big]$$

In [5]:
function Base.:*(f::Dual{T}, g::Dual{T}) where {T}
    return Dual{T}(f.val*g.val, f.val*g.grad+f.grad*g.val)
end

function Base.:*(f::Number, g::Dual{T}) where {T}
    return Dual{T}(f*g.val, f*g.grad)
end

function Base.:*(f::Dual{T}, g::Number) where {T}
    return Dual{T}(f.val*g, f.grad*g)
end

- **Division Rule**

    $$\bigg(\frac{f}{g}\bigg)(x)\rightarrow \frac{f(x)}{g(x)}+\epsilon \bigg[\frac{f^{'}(x)g(x)-f(x)g^{'}(x)}{g(x)^2}\bigg]$$

In [6]:
function Base.:/(f::Dual{T}, g::Dual{T}) where {T}
    return Dual{T}(f.val/g.val, (f.grad*g.val-f.val*g.grad)/g.val^2)
end

function Base.:/(f::Number, g::Dual{T}) where {T}
    return Dual{T}(f/g.val, (-f*g.grad)/g.val^2)
end

function Base.:/(f::Dual{T}, g::Number) where {T}
    return Dual{T}(f.val/g, (f.grad*g)/g^2)
end

- **Exponential Operator**

    We know that $x^3$ basically means $((x\times x)\times x)$, so we can exploit the **multiplication rule** to define this.

In [7]:
function Base.:^(f::Dual{T}, g::Number) where {T}
    return Base.power_by_squaring(f, g)
end

## Example: Differentiating a simple equation
Suppose we have a function $f(x)=x^2+2x$. From chain rule, we can write $\frac{\partial f}{\partial x}=\frac{\partial f}{\partial x} \times \frac{\partial x}{\partial x}$ where $\frac{\partial x}{\partial x}=1$. Hence to get the *automatic differentiation* to supply $\frac{\partial f}{\partial x}$, we need to define the input $x$ as a `Dual` number, i.e. `Dual(x, 1)`, where `x` is the value at which we want to compute gradient.

Let's evaluate gradient at $x=2$.

In [8]:
# Defining Function
h(x) = x^2 + 2*x

# Defining x
x = Dual(2,1)

# Evaluating h(x)
y = h(x)

println("Function Value: ", y.val)
println("Function Gradient: ", y.grad)

Function Value: 8
Function Gradient: 6


The amazing thing is that due to Vector Registers in Modern CPUs, value and grad are evaluated in parallel. This is also known as Vectorisation and is very common in other libraries like NumPy!

In [9]:
# Normal Function eval
println("Just Evaluating value of the Function")
@btime _ = h(2);

# Dual Function eval
println("Evaluating Value and Derivative of the function")
@btime _ = h(Dual(2,1));

Just Evaluating value of the Function


  0.929 ns (0 allocations: 0 bytes)
Evaluating Value and Derivative of the function


  0.929 ns (0 allocations: 0 bytes)


Suppose we have functions $f(x, y) = x^2 + xy$ and $g(x, y) = y^3 + x$ and want to compute jacobian 
$J = 
\begin{bmatrix} 
\frac{\partial f}{\partial x} & \frac{\partial f}{\partial y}\\
\frac{\partial g}{\partial x} & \frac{\partial g}{\partial y}
\end{bmatrix}
$ at $x=3, y=4$.

This can be done by evaluating all four elements separately as follows:

In [10]:
# Defining Functions
f(x, y) = x^2 + x*y
g(x, y) = y^3 + x

# Getting Derivatives
df_dx = f(Dual(3, 1), Dual(4, 0)).grad
df_dy = f(Dual(3, 0), Dual(4, 1)).grad

dg_dx = g(Dual(3, 1), Dual(4, 0)).grad
dg_dy = g(Dual(3, 0), Dual(4, 1)).grad


J = zeros(2, 2)
J[1,1] = df_dx
J[1,2] = df_dy
J[2,1] = dg_dx
J[2,2] = dg_dy

println("Jacobian: ")
show(stdout, "text/plain", J)

Jacobian: 


2×2 Matrix{Float64}:
 10.0   3.0
  1.0  48.0

Hence, for $f(x_1, x_2, \cdots, x_n)$, we'll have to do differentiation `n` times! Another problem here is that we are not able to utilise the Vectorisation (i.e. Jacobian Computation is not parallel). We know that 
$\nabla f = \left[\begin{array}{ccc}
\dfrac{\partial f(x,y)}{\partial x} & \dfrac{\partial f(x,y)}{\partial y}
\end{array}\right]$, and if we want vectorisation we need something like `df = ff(X).grads where (X=[x y])` and `df=[ff(Dual(3, 1), Dual(4, 0)).grad, ff(Dual(3, 0), Dual(4, 1)).grad]`, in turn utilise the **vector instructions** to calculate each partial derivative in parallel.

This can be done by storing the derivatives in a `Vector` type rather than a scalar `grad`.

## `MultiDual`
As discussed above, to utilise vectorisation we can store gradients in a `SVector` type. This is a Static Vector which lives on the stack for fast access (this makes sense because the dimensions for most real life problems can't be horribly large). In `MultiDual`, `N` defines the number of variables in the function and `T` defines the type (`Int`, `Float`, etc. etc.).

In [11]:
struct MultiDual{N,T}
	# Value
	val::T
	# SVector is static vector which lives on the stack
	grads::SVector{N,T} 
end

Various rules can be defined using Taylor Series (like before), i.e. if $x$ is a vector then Taylor Series is defined as 

$$f(x+\epsilon)=f(x)+\epsilon \nabla f(x)+O(\epsilon)$$

Only change is that $f^{'}$ is replaced by $\nabla f$ and the same thing goes for various differentiation rules.

- **Addition Rule**
    $$(f+g)(x)\rightarrow f(x)+g(x)+\epsilon \big[\nabla f(x)+\nabla g(x)\big]$$

In [12]:
function Base.:+(f::MultiDual{N,T}, g::MultiDual{N,T}) where {N,T}
    return MultiDual{N,T}(f.val+g.val, f.grads+g.grads)
end

function Base.:+(f::Number, g::MultiDual{N,T}) where {N,T}
    return MultiDual{N,T}(f+g.val, g.grads)
end

function Base.:+(f::MultiDual{N,T}, g::Number) where {N,T}
    return MultiDual{N,T}(f.val+g, f.grads)
end

- **Subtraction Rule**
    $$(f-g)(x)\rightarrow f(x)-g(x)+\epsilon \big[\nabla f(x)-\nabla g(x)\big]$$

In [13]:
function Base.:-(f::MultiDual{N,T}, g::MultiDual{N,T}) where {N,T}
    return MultiDual{N,T}(f.val-g.val, f.grads-g.grads)
end

function Base.:-(f::Number, g::MultiDual{N,T}) where {N,T}
    return MultiDual{N,T}(f-g.val, g.grads)
end

function Base.:-(f::MultiDual{N,T}, g::Number) where {N,T}
    return MultiDual{N,T}(f.val-g, f.grads)
end

- **Multiplication Rule**
    $$(f \cdot g)(x)\rightarrow f(x)g(x)+\epsilon \big[f(x)\nabla g(x)+\nabla f(x)g(x)\big]$$

In [14]:
function Base.:*(f::MultiDual{N,T}, g::MultiDual{N,T}) where {N,T}
    return MultiDual{N,T}(f.val*g.val, f.val*g.grads+f.grads*g.val)
end

function Base.:*(f::Number, g::MultiDual{N,T}) where {N,T}
    return MultiDual{N,T}(f*g.val, f*g.grads)
end

function Base.:*(f::MultiDual{N,T}, g::Number) where {N,T}
    return MultiDual{N,T}(f.val*g, f.grads*g)
end

- **Division Rule**
    $$(\frac{f}{g})(x)\rightarrow \frac{f(x)}{g(x)}+\epsilon \bigg[\frac{\nabla f(x)g(x)-f(x)\nabla g(x)}{g(x)^2}\bigg]$$

In [15]:
function Base.:/(f::MultiDual{N,T}, g::MultiDual{N,T}) where {N,T}
    return MultiDual{N,T}(f.val/g.val, (f.grads*g.val-f.val*g.grads)/g.val^2)
end

function Base.:/(f::Number, g::MultiDual{N,T}) where {N,T}
    return MultiDual{N,T}(f/g.val, (-f.val*g.grads)/g.val^2)
end

function Base.:/(f::MultiDual{N,T}, g::Number) where {N,T}
    return MultiDual{N,T}(f.val/g, (f.grads*g.val)/g^2)
end

- **Exponential operator**

In [16]:
function Base.:^(f::MultiDual{N,T}, g::Number) where {N,T}
    return Base.power_by_squaring(f,g)
end

## Example: Efficient Jacobian
Let's now compute the Jacobian using `MultiDual`. The two functions are 

$$f(x, y) = x^2 + xy$$ 

$$g(x, y) = y^3 + x$$

The next step is to define `MultiDual` functions for `x` and `y`. 
- Let a function $xx(x,y)=x + 0 \times y$. `MultiDual` `xx` can be written as `xx = MultiDual(x, SVector(1.,0.))`. `SVector(1.,0.)` is written because $\frac{\partial xx}{x} = 1$ and $\frac{\partial xx}{y} = 0$, hence these are set as the first and second elements of the vector.
- Similar for $yy(x,y)=y + 0 \times x$, `yy=MultiDual(y, SVector(0.,1.))`.

Jacobian can then easily be calculated in parallel.

In [17]:
f(x, y) = x^2 + x*y
g(x, y) = y^3 + x

# Jacobian at x=3 and y=4
xx = MultiDual(3., SVector(1.,0.))
yy = MultiDual(4., SVector(0.,1.))

# Function and Jacobian computation
f_ = f(xx, yy)
g_ = g(xx, yy)

df_dx, df_dy = f_.grads 
dg_dx, dg_dy = g_.grads 

J = zeros(2, 2)
J[1,1] = df_dx
J[1,2] = df_dy
J[2,1] = dg_dx
J[2,2] = dg_dy

println("Jacobian: ")
show(stdout, "text/plain", J)

Jacobian: 
2×2 Matrix{Float64}:
 10.0   3.0
  1.0  48.0

## Parallel Forward Mode Automatic Differentiation: Stencil Computation
So far the derivative computation has been parallelised. However, if the computations have a structure that can be further parallelised, it can be made even more efficient. To demonstrate this, let's consider an example of 3 point Stencil Computation. Taking an input array `in_arr` of length `n`, then the 3-point stencil computation has the following pseudocode:

```
for in_arr_index = 2 to n-1
    out_arr[in_arr_index - 1] = a*in_arr[in_arr_index - 1] + b*in_arr[in_arr_index] + c*in_arr[in_arr_index + 1]
end
```

In the above calculations, `a`, `b` and `c` are scalars and I've demonstrated using the multiplication and addition operations. However, that can be changed to encorporate any other operation. For instance, to make things a little more interesting, I'm replacing addition with multiplication as well. Parallelising this is just a matter of running the loop interations over multiple threads in parallel. As each thread will write to a unique location, there are no race conditions! 

In [26]:
# Stencil Computation
function stencil(a,b,c) 
    return 2*a * 3*b * 4*c
end

# Parallel Stencil Computation over an array
function parallel_stencil(arr_in, vals_out, grads_out)
    Threads.@threads for i=2:size(arr_in)[1]-1
        output = stencil(MultiDual(arr_in[i-1], SVector(1.,0.,0.)), MultiDual(arr_in[i], SVector(0.,1.,0.)), MultiDual(arr_in[i+1], SVector(0.,0.,1.)))
        vals_out[i-1] = output.val
        grads_out[i-1,:] = output.grads
    end

    return vals_out, grads_out
end

parallel_stencil (generic function with 2 methods)

In [36]:
# Number of available threads
println("Number of Threads: ", Base.Threads.nthreads())

# Computation
arr_in = rand(100000,1)
vals_out = zeros(size(arr_in)[1]-2,1)
grads_out = zeros(size(arr_in)[1]-2,3)

vals_out, grads_out = parallel_stencil(arr_in, vals_out, grads_out) 

println("Parallel Evaluation Time")
@btime _, _ = parallel_stencil(arr_in); 

Number of Threads: 16
Parallel Evaluation Time


  75.110 μs (98 allocations: 3.06 MiB)


Just to ensure that we are actually getting a speedup, let's look at the sequential stencil computation for a very large array size.

In [37]:
# Sequential Stencil Computation over an array
function sequential_stencil(arr_in, vals_out, grads_out)
    for i=2:size(arr_in)[1]-1
        output = stencil(MultiDual(arr_in[i-1], SVector(1.,0.,0.)), MultiDual(arr_in[i], SVector(0.,1.,0.)), MultiDual(arr_in[i+1], SVector(0.,0.,1.)))
        vals_out[i-1] = output.val
        grads_out[i-1,:] = output.grads
    end

    return vals_out, grads_out
end

sequential_stencil (generic function with 2 methods)

In [38]:
println("Sequential Evaluation Time")
@btime _, _ = sequential_stencil(arr_in, vals_out, grads_out); 

Sequential Evaluation Time


  283.019 μs (1 allocation: 32 bytes)


We can perform several modifications to this stencil computation but in essence, the runtimes differences will be very similar. From this, we can see that the big speedup can only be achieved if the computations can be inherently parallelised. Parallel Gradient Computation does not impact run times that much as the variables can't be horribly large. Hence it's wise to not call threads and just utilise the vector processes to parallelise the gradient computation and only use threads to distribute the computations in the graph across multiple threads.